In [79]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pickle
import gc
import xgboost as xgb
from sklearn.calibration import CalibratedClassifierCV
from os import path
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

import os
import re
from nltk.corpus import stopwords
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

from fuzzywuzzy import fuzz
#import spacy

ModuleNotFoundError: No module named 'distance'

In [80]:
!pip install distance

  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16276 sha256=ee990ef928b89c53cd3282b065fa8c43d14b3f853584f5492cdecfd25f86ea84
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\fb\b3\aa\04241cced6d1722b132273b1d6aafba317887ec004f48b853a
Successfully built distance


In [37]:
pred_data = pd.read_csv("../data/pred_data20.csv")

In [38]:
pred_data

,id,qid1,qid2,question1,question2,is_duplicate
0,6028,11823,11824,Who is the most powerful person in earth and why?,Who actually is the most powerful person or wh...,1
1,139097,23573,34861,How can I make a suicide look like an accident?,How do I die and make it look like an accident?,1
2,29616,54775,54776,Which Jedi Knights used red-colored lightsabers?,Who was the first Jedi Knight?,0
3,160739,68835,250621,"What is the quickest way to make $8,000?",What are the quickest ways to make money?,1
4,354430,99603,483552,What are some good deodorants and perfumes for...,What is the average percentage of water that i...,0
5,281504,101771,30183,How can I come over borderline personality dis...,"What is borderline personality disorder, and h...",1
6,66640,115474,115475,If you could go back in time and give your you...,"If you could go back in time, what advice woul...",1
7,130187,117142,50267,What are the best new products in technology t...,What are some mind blowing gadgets that exist ...,1
8,78043,133143,133144,How do I impress a girl whom I like?,How do I impress the girl on whom I have a crush?,1
9,78842,134351,134352,Are sex and love different?,Is sex and love different?,1


In [39]:
X_pred = pred_data.iloc[:,:-1]
y_pred = pred_data.iloc[:,-1]

In [48]:
def load_model(model_path:str):
    with open(model_path, "rb") as m:
        model = pickle.load(m)
    return model

In [ ]:
trained_model = load_model("../Models/xgboost4_calibrated.pickle")

In [ ]:
data['freq_qid1'] = data.groupby('qid1')['qid1'].transform('count')
data['freq_qid2'] = data.groupby('qid2')['qid2'].transform('count')
data['q1len'] = data['question1'].str.len()
data['q2len'] = data['question2'].str.len()
data['q1_n_words'] = data['question1'].apply(lambda row: len(row.split(" ")))
data['q2_n_words'] = data['question2'].apply(lambda row: len(row.split(" ")))

def normalized_word_Common(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
    return 1.0 * len(w1 & w2)
data['word_Common'] = data.apply(normalized_word_Common, axis=1)

def normalized_word_Total(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
    return 1.0*(len(w1) + len(w2))
data['word_Total'] = data.apply(normalized_word_Total, axis=1)

def normalized_word_share(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
    return 1.0 * len(w1 & w2)/(len(w1) + len(w2))

data['word_share'] = data.apply(normalized_word_share, axis=1)
data['freq_q1+q2'] = data['freq_qid1'] + data['freq_qid2']
data['freq_q1-q2'] = abs(data['freq_qid1'] - data['freq_qid2'])

data.head()

In [ ]:
SAFE_DIV = 0.0001 # to get result in 4 decimal points

STOP_WORDS = stopwords.words('english')

def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)

    porter = PorterStemmer()
    pattern = re.compile('\W')

    if type(x) == type(''):
        x = re.sub(pattern,' ',x)

    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()

    return x

In [ ]:
def get_token_features(q1, q2):
    token_features = [0.0]*10

    # Converting questions into tokens
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens)== 0 or len(q2_tokens)==0:
        return token_features

    # Getting non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOPWORDS])
    q2_words = set([word for word in q2_tokens if word not in STOPWORDS])

    # Getting stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOPWORDS])
    q2_stops = set([word for word in q2_tokens if word in STOPWORDS])

    # Getting the common non-stopwords from questions pair
    common_word_count = len(set(q1_words).intersection(set(q2_words)))

    # Getting the common stopwords from questions pair
    common_stop_count = len(set(q1_stops).intersection(set(q2_stops)))

    # Getting the common non-stopwords from questions pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))

    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)

    # Last word of both questions is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])

    # First word of both questions is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])

    token_features[8] = abs(len(q1_tokens) - len(q2_tokens))

    # Average token length of both Questions
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2

    return token_features

def get_longest_substr_ratio(a,b):
    strs = list(distance.lcsubstrings(a,b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b))+1)

def extract_features(df):
    # preprocessing each question
    df['question1'] = df["question1"].fillna("").apply(preprocess)
    df['question2'] = df["question2"].fillna("").apply(preprocess)

    print("Token features..........")

    # Merging features with dataset
    token_features = df.apply(lambda x: get_token_features(x['question1'], x['question2']), axis=1)

    df["cwc_min"]       = list(map(lambda x:x[0], token_features))
    df["cwc_max"]       = list(map(lambda x:x[1], token_features))
    df["csc_min"]       = list(map(lambda x:x[2], token_features))
    df["csc_max"]       = list(map(lambda x:x[3], token_features))
    df["ctc_min"]       = list(map(lambda x:x[4], token_features))
    df["ctc_max"]       = list(map(lambda x:x[5], token_features))
    df["last_word_eq"]  = list(map(lambda x:x[6], token_features))
    df["first_word_eq"] = list(map(lambda x:x[7], token_features))
    df["abs_len_diff"]  = list(map(lambda x:x[8], token_features))
    df["mean_len"]      = list(map(lambda x:x[9], token_features))

    ## Computing fuzzy features
    print("Fuzzy Features........")

    df['token_set_ratio'] = df.apply(lambda x: fuzz.token_set_ratio(x['question1'], x['question2']),axis=1)
    df['token_sort_ratio'] = df.apply(lambda x: fuzz.token_sort_ratio(x['question1'], x['question2']),axis=1)
    df['fuzz_ratio'] = df.apply(lambda x: fuzz.QRatio(x['question1'], x['question2']),axis=1)
    df['fuzz_partial_ratio'] = df.apply(lambda x: fuzz.partial_ratio(x['question1'], x['question2']),axis=1)
    df['longest_substr_ratio'] = df.apply(lambda x: get_longest_substr_ratio(x['question1'], x['question2']),axis=1)
    return df


In [ ]:
stopwords = set(STOPWORDS)

stopwords.add("said")
stopwords.add("br")
stopwords.add(" ")
stopwords.remove("not")
stopwords.remove("no")
stopwords.remove("like")

In [ ]:
word2tfidf = pickle.load()

In [ ]:
nlp = spacy.load('en_core_web_lg')

vecs1 = []
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1)
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)]) 

    for word1 in doc1:
        vec1 = word1.vector

        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        mean_vec1 += vec1*idf

    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)

df["q1_feats_m"] = list(vecs1)

